# Notebook for Downloading Events of a Season

### Imports

In [ ]:
import logging
import csv
from typing import Any, Dict, List
import json
import json
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

from sportradar_datacore_api.handball import HandballAPI

In [ ]:
load_dotenv()  # Load environment variables from .env file if present


### Configuration

In [ ]:
NAME_COMPETITION = "1. Handball-Bundesliga"

NAME_SEASON = "DAIKIN HBL 2024/25"
YEAR_SEASON = int(NAME_SEASON.split()[-1].split("/")[0])

PATH_TO_OUTPUT = os.path.join(
    os.getcwd(), "..", "data", "season_24_25"
)

# create path if it does not exist
os.makedirs(PATH_TO_OUTPUT, exist_ok=True)

### Initialize API

In [ ]:
api = HandballAPI(
    base_url=os.getenv("BASE_URL", ""),
    auth_url=os.getenv("AUTH_URL", ""),
    client_id=os.getenv("CLIENT_ID", ""),
    client_secret=os.getenv("CLIENT_SECRET", ""),
    org_id=os.getenv("CLIENT_ORGANIZATION_ID"),
    scopes=["read:organization"],
    sport="handball",
)

### Get wanted competition ID

In [ ]:
id_competition = api.get_competition_id_by_name(NAME_COMPETITION)

# id_competition = int(id_competition)

# Check if the competition was found
if not id_competition:
    raise ValueError(f"Competition '{NAME_COMPETITION}' not found.")
else:
    print(f"→ Competition '{NAME_COMPETITION}' -> {id_competition}")

### Get wanted season ID

In [ ]:
id_season = api.get_season_id_by_year(
        competition_id=id_competition, season_year=YEAR_SEASON
    )
# Check if the season was found
if not id_season:
    raise ValueError(f"Season '{NAME_SEASON}' not found in competition '{NAME_COMPETITION}'.")
else:
    print(f"→ Season '{NAME_SEASON}' -> {id_season}")

### Get the fixtures (matches) of a season

In [ ]:
list_fixtures = api.get_list_matches_by_season_id(season_id=id_season)

print(f"Found {len(list_fixtures)} fixtures.")
# Print the first fixture in a pretty format
print(json.dumps(list_fixtures[0], indent=2, default=str))


### Get fixture

In [ ]:
events = api.get_fixture_events_by_id(list_fixtures[0].fixture_id)
print(f"  Events count: {len(events)}")

### Save to CSV

In [ ]:
api.save_events_to_csv(
    events,
    file_path=f"fixture_{list_fixtures[0].fixture_id}_events.csv",
)